# Data Wrangling with MongoDB
### Thuy Quach



Map area: Seattle, WA, United States

http://www.openstreetmap.org/relation/237385

https://mapzen.com/data/metro-extracts/#seattle

Abstract:

Open Street Map is a collaborative project to create a free editable map of the world. It is a great source of data. However as any open source data, it contains many inconsistent or incorrect information such as street types, postcodes and city names. The tasks of the project was to audit, fixing and processing the dataset of Seattle. MongoDB queries were used to obtain the overview of the data is and also other investigations. 

### 1. Parse the download oms file and take a sample part of the map

The size of original seattle.osm was 1.54GB. To make it run with my PC, I parsed the file and took a sample of 1/20th of it. 

In [18]:
# parse file and make a sample file
import xml.etree.cElementTree as ET
OMS_FILE = "seattle_washington.osm"
SAMPLE_FILE = "seattle_sample.osm"
def get_element(oms_file, tags = ('node', 'way', 'relation')):
    context = ET.iterparse(oms_file, events = ('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n')
    
    #write every 100th top level element
    for i, element in enumerate(get_element(OMS_FILE)):
        if i%20 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))
    output.write('</osm>')

In [71]:
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [299]:
import warnings
warnings.filterwarnings('ignore')

### 2. Problems encountered in the map

#### Check problematic characters 

Before process the data and add it into MongoDB, we should check the "k"
value for each "tag" and see if they can be valid keys in MongoDB, as well as
see if there are any other potential problems.

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
"""
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        k = element.get("k")
        #print k
        if re.search(lower, k):
            keys['lower'] += 1
        elif re.search(lower_colon, k):
            keys['lower_colon'] += 1
        elif re.search(problemchars, k):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
        pass
        
    return keys




def process_map_key_type(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [314]:
process_map_key_type('seattle_sample.osm')

{'lower': 102074, 'lower_colon': 114390, 'other': 3692, 'problemchars': 0}

- There are no tags with problematic characters.
- There are 102074 tags that contain only lowercase letters and are valid.
- There are 114390 for otherwise valid tags with a colon in their names.
- There are  3692 tags that do not fall into the other three categories.

#### Problems with street types

We need to  audit the OSMFILE to see the street types are appropriate ones.

In [2]:
"""
audit the OSMFILE reflect the changes needed to fix the unexpected street types to the appropriate ones in the expected list.
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    #print street_types
    return street_types
            


In [4]:
print "Here is the list of street types:" + "\n" + str(audit('seattle_sample.osm').keys())

Here is the list of street types:
['Glen', 'Fir', 'Northeast', 'Ridge', 'West', 'St.', 'Heights', 'Rd', 'Speedway', 'street', 'Blackburn', 'Northwest', 'Way', '1st', 'Gate', 'Circle', 'East', 'avenue', 'Meadows', 'Highway', 'Southwest', '104', 'Cleveland', 'North', 'west', 'Rise', 'Reach', 'NE', 'Meridian', 'Southeast', '18th', 'Loop', 'Hwy', '9', 'Snoqualmie', '25th', 'Section', 'NW', 'Laventure', 'E', 'Center', 'Sandalwood', '36th', 'Plaza', 'Alley', 'St', '99', 'Division', 'S', 'Green', 'W', 'Close', '20', 'N', 'Estates', 'Gardens', 'South', 'Point', 'S.E.', 'Terrace', 'SW', 'Esplanade', 'r', 'Crescent', 'Waugh', 'Broadway', 'southwest', 'SE']


We could see that the street types were over-abbreviated such as "S.E", "N", "S", etc. We need to fix the unexpected street names to the appropriate ones by using mapping dictionary and then updated the street name. We will later use it to update the street names in json data.

In [25]:
# Update street names
mapping = { "SE": "South East",
            "St.": "Street", 
            "Rd": "Road",
            "SouthEast" : "South East",
            "Rd.": "Road",
            "Northeast": "North East",
            "Northwest" : "North West",
            "Southwest" : "South West",
            "N" : "North",
            "E" : "East",
            "W" : "West",
            "S" : "South",
           "S.E": "South East",
           "SW" : "South West"
            }

def update_name(name, mapping):
    for word in mapping.keys():
        name = re.sub(r'\b' + word + r'\b\.?',  mapping[word], name)
    return name
            

#### Problems with postcode

Postcode is also important data. All the postcodes in Seattle start with 981. Let's check to see all the postcode are correct.

In [6]:
"""
audit the OSMFILE reflect the changes needed to fix the unexpected postcode types to the appropriate ones in the expected list.
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

# check if the first three characters start with 981
postcode_re = re.compile(r'^981')

# initiate the not match postcode
postcode_diff = []

def audit_postcode(postcode_re, postcode_num):
    if not postcode_re.match(postcode_num):
        if postcode_num not in postcode_diff:
            postcode_diff.append(postcode_num)


def is_postcode_num(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit_postcode_name(osmfile):
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postcode_num(tag):   
                    audit_postcode(postcode_re, tag.attrib['v'])
    osm_file.close()
    return postcode_diff
            


In [86]:
len(audit_postcode_name("seattle_sample.osm"))

128

- There are 128 unmatched postcode.

In [7]:
print "Some of ummacthed postcode: " + "\n" + str(audit_postcode_name("seattle_example.osm")[:10])

Some of ummacthed postcode: 
['98070', 'V8X 4V1', '98004', '98005', '98273', '98503', '98011', '98057', '98002', '98052']


- Quick checking some of the postcodes are in other cities next to Seattle such as Kirkland and Bellevue. It would be interesting to find out later how many of the data are from those cities using MongoDB.

#### Problem with city name

From the postcode data, there was some data in different cities than Seattle in the dataset. I will later check with MongoDB queries to see more details.

### 3. Prepare the data set for MongoDB

#### Process the oms to json

The oms file was parsed. The output data was shaped by its elements and then wrote on json file. The json file then was imported to MongoDB for analyzing.

In [22]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    created ={}
    if element.tag == "node" or element.tag == "way" :
        
        for e in element.attrib.keys():
            if e in CREATED:
                created[e] = element.get(e)
                node["created"] = created
        
            
        node["id"] = element.get('id')
        node["type"] = element.tag
        node["visible"] = element.get('visible')
        pos = [element.get('lat'), element.get('lon')]
        for i in pos:
            if type(i) == str:
                node['pos'] = [float(i) for i in pos]              
                          
        address = {}
        node_refs = []
            
        for tag in element:
            
            if tag.tag == "tag":
                k = tag.get('k')
                v = tag.get('v')
                if re.search('problemchars', k):
                    pass
                elif re.search(r'\w+:\w+:\w+', k):
                    pass
                elif k.startswith('addr:'): 
                    address[k[5:]] = v
                    node['address'] = address
                else:
                    node[k] = v
            

            if tag.tag == 'nd':
                ref = tag.get('ref')
                #print type(ref)
                node_refs.append(tag.attrib['ref'])
                node['node_refs'] = node_refs
        
        return node
    
    else:
        return None
    



def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [293]:
data = process_map('seattle_sample.osm', False)

#### Auditting data with MongoDB

##### Check the postcode

MongoDB queries were used to analyze top postcodes. 

In [315]:
# import mongoDB 
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['test']

# return the aggregation
# the collection name seattle1
def aggregate(db, pipeline):
    return [doc for doc in db.seattle2.aggregate(pipeline)]


In [12]:
postcode_queries = [{"$match": {"address.postcode": {"$exists": 1}}},
                   {"$group": {"_id": "$address.postcode",
                              "count": {"$sum":1}}},
                   {"$sort": {"count": -1}}]
result = aggregate(db, postcode_queries)
# top 10 results

print "Top 10 postcode:" + "\n" + str(result[0:10])

Top 10 postcode:
[{u'count': 1147, u'_id': u'98034'}, {u'count': 968, u'_id': u'98033'}, {u'count': 905, u'_id': u'98115'}, {u'count': 838, u'_id': u'98103'}, {u'count': 716, u'_id': u'98118'}, {u'count': 692, u'_id': u'98117'}, {u'count': 610, u'_id': u'98125'}, {u'count': 492, u'_id': u'98105'}, {u'count': 470, u'_id': u'98108'}, {u'count': 463, u'_id': u'98144'}]


Out of top 10 postcodes, two are in Kirkland (start with 980) and the rest are in Seattle (start with 981). 

In [18]:
# all data with postcode
postcode_all = [{"$match": {"address.postcode": {"$exists": 1}}}]
result = aggregate(db, postcode_all)
print "Total data with postcode is " + str(len(result)) + " out of " + str(db.seattle1.find().count()) + " data."

Total data with postcode is 13219 out of 378925 data.


In [316]:
# data with Seattle postcode
postcode_Seattle = [{"$match": {"address.postcode": {"$exists": 1, '$regex': '^981'}}}]
result = aggregate(db, postcode_Seattle)
print "Total data with Seattle postcode is " + str(len(result))

Total data with Seattle postcode is 10162


There are many data don't have postcode. Among them, most are Seattle zipcode. Therefore we can remove the data that has postcode unmatched '981'. 

##### Check the city name

MongoDB queries were used to analyze all city names.

In [317]:
# city names
city_queries = [{"$match": {"address.city": {"$exists": 1}}},
                   {"$group": {"_id": "$address.city",
                              "count": {"$sum":1}}},
                   {"$sort": {"count": -1}}]
result_city = aggregate(db, city_queries)
print "Top 10 cities: " + str(result_city[:10]) + "\n"

print "Number of cities: " + str(len(result_city))


Top 10 cities: [{u'count': 10143, u'_id': u'Seattle'}, {u'count': 2113, u'_id': u'Kirkland'}, {u'count': 606, u'_id': u'Saanich'}, {u'count': 559, u'_id': u'Mount Vernon'}, {u'count': 138, u'_id': u'Langford'}, {u'count': 115, u'_id': u'Oak Bay'}, {u'count': 92, u'_id': u'Colwood'}, {u'count': 79, u'_id': u'Esquimalt'}, {u'count': 78, u'_id': u'Sooke'}, {u'count': 49, u'_id': u'Metchosin'}]

Number of cities: 82


Some cites are not from Seattle such as Kirkland, Saanich, Langford, Oak Bay etc. Kirkland is a neighbor city of Seattle. Saanich and Oak Bay are two cities in Victoria, Canada. They are both nearby cities to Seattle. There are total of 81 cities are not Seattle.

#### Data cleaning

So, after running the analysis, we need to clean the street types, postcode and city names. 

##### Update street name

In [294]:
# Update street name
def update_streetname(data):
    for entry in data:
        if 'address' in entry.keys() and 'street' in entry['address']:
            name = entry['address']['street']
            entry['address']['street'] = update_name(name, mapping)
    return data

clean_name_data = update_streetname(data)
print "Total data after updating street types: " + str(len(clean_name_data))

Total data after updating street types: 378925


##### Drop data with postcode unmatch '981'

In [295]:
# drop data with postcode unmatch 981

def remove_postcode(data):
    returned_data = []
    for entry in data:
        if 'address' in entry.keys() and 'postcode' in entry['address'] and not entry['address']['postcode'].startswith('981'):
            continue
        else:
            returned_data.append(entry)    
    return returned_data
            
clean_postcode_data = remove_postcode(clean_name_data)
print "Total data after updating street types and postcode: " + str(len(clean_postcode_data))

Total data after updating street types and postcode: 375868


##### Drop city name unmatched 'Seattle' 

In [296]:
# drop city with name different with Seattle

def remove_cityname(data):
    returned_data = []
    for entry in data:
        if 'address' in entry.keys() and 'city' in entry['address'] and entry['address']['city'].lower() != 'seattle':
            continue
        else:
            returned_data.append(entry)    
    return returned_data
            
updated_data = remove_cityname(clean_postcode_data)
print "Total data after updating street types, postcode and city names : " + str(len(updated_data))

Total data after updating street types, postcode and city names : 374540


#### Write clean data to json

In [327]:
# write data to json
with codecs.open('clean_seattle3.json', 'w') as f:
    f.write(json.dumps(updated_data)+ "\n")



#### Load the data into Mongo DB

In [328]:

import json
import pymongo

# Get database connection db first

with open('clean_seattle3.json', 'rb') as json_file:
    for line in json_file:
        db.clean_seattle3.insert(json.loads(line))

### 4. Data Overview

This section contains basic statistics about the dataset. MongoDB queries are used to get the data.

In [329]:
# load the data
def aggregate_clean(db, pipeline):
    return [doc for doc in db.clean_seattle3.aggregate(pipeline)]

In [332]:
import os
print "File name : clean_seattle3.json " + "\n" + "File size: " + str(os.path.getsize('clean_seattle3.json')/1000000) + "MB"

File name : clean_seattle3.json 
File size: 91MB


In [334]:
# number of documents
print "Number of documents: " + str(db.clean_seattle3.find().count())

Number of documents: 374540


In [335]:
# number of node
print "Number of nodes: " + str(db.clean_seattle3.find({"type": "node"}).count())

Number of nodes: 343065


In [336]:
# number of ways
print "Number of ways: " + str(db.clean_seattle2.find({"type": "way"}).count())

Number of ways: 94413


In [337]:
unique_users =    [
                   {"$group": {"_id": "$created.user",
                              "count": {"$sum":1}}},
                   {"$sort": {"count": -1}}]


print "Number of unique users: " + str(len(aggregate_clean(db, unique_users)))

Number of unique users: 1447


In [338]:
# top contributing users

print ("Top 1st user: " + str(aggregate_clean(db, unique_users)[0]))


Top 1st user: {u'count': 61392, u'_id': u'Glassman'}


In [339]:
# top 10 contributing users

print ("Top 10th user: " + "\n" + str(aggregate_clean(db, unique_users)[0:10]))

Top 10th user: 
[{u'count': 61392, u'_id': u'Glassman'}, {u'count': 37383, u'_id': u'SeattleImport'}, {u'count': 33100, u'_id': u'tylerritchie'}, {u'count': 31165, u'_id': u'woodpeck_fixbot'}, {u'count': 16124, u'_id': u'alester'}, {u'count': 11278, u'_id': u'STBrenden'}, {u'count': 10735, u'_id': u'Glassman_Import'}, {u'count': 8995, u'_id': u'Brad Meteor'}, {u'count': 8441, u'_id': u'Amoebabadass'}, {u'count': 6041, u'_id': u'zephyr'}]


### 5. Additional analysis

#### Improving OMS data quality suggestions

##### Address with over-abbreviated and inconsistent street types

The street types data could be improved by let the contributing users enter an auto-corrected street types. Each countries and/or cities may have different street types. If possible, the street type data of interested area could be collected via cross-reference sources such as local transit goverment, postmail company or Google Maps. In case there is no standard existing street types, the local open street map community could make one data processor similar to my auditing and updating street types functions.

##### Unmatched postcode and city name

Similar to street types data, postcode data could be much improved by having an standard postcode. If the new data's postcode does not in the appropriate list, it should be warned. Same with city name, it should be same with city of interest. A data processor similar to my auditing and fixing postcode and city name could work well.

##### Encourage user parcipation and collaboration

From the user data, there are 1447 users in which top 10 user contribution is 60%. In top 10 user there is only one name appears with the word 'bot'. It looks like it could have a great chance of collaboration to make and edit beter map. Certain gamification elements, such as badges or leaderboard would strive to leverage user's desires to create better data and community collaboration.

#### Additional data exploration using MongoDB queries

##### Top 10 amenities

In [340]:
top10_amenities = [{"$match": {"amenity": {"$exists": 1}}},
                   {"$group": {"_id": "$amenity",
                              "count": {"$sum":1}}},
                   {"$sort": {"count": -1}},
                   {"$limit": 10}]
aggregate_clean(db, top10_amenities)

[{u'_id': u'parking', u'count': 383},
 {u'_id': u'bicycle_parking', u'count': 149},
 {u'_id': u'school', u'count': 123},
 {u'_id': u'restaurant', u'count': 117},
 {u'_id': u'bench', u'count': 94},
 {u'_id': u'place_of_worship', u'count': 69},
 {u'_id': u'cafe', u'count': 53},
 {u'_id': u'waste_basket', u'count': 48},
 {u'_id': u'fuel', u'count': 43},
 {u'_id': u'fast_food', u'count': 43}]

We could see that 'parking' is the top amenity in Seattle. It is not a surprise given Seattle is a home city of many companies. 'Bicycle_parking', 'bench' and 'waste_basket' are also in the top 10 amenity. There was also a lot of cafe. It could be Seattle is home of Starbucks.

##### Top 10 cuisine

In [341]:
top10_cuisine = [{"$match": {"amenity": {"$exists": 1}, "amenity": "restaurant"}},
                   {"$group": {"_id": "$cuisine",
                              "count": {"$sum":1}}},
                   {"$sort": {"count": -1}},
                   {"$limit": 10}]
aggregate_clean(db, top10_cuisine)

[{u'_id': None, u'count': 49},
 {u'_id': u'pizza', u'count': 9},
 {u'_id': u'american', u'count': 7},
 {u'_id': u'mexican', u'count': 6},
 {u'_id': u'chinese', u'count': 5},
 {u'_id': u'thai', u'count': 4},
 {u'_id': u'sandwich', u'count': 4},
 {u'_id': u'italian', u'count': 4},
 {u'_id': u'japanese', u'count': 3},
 {u'_id': u'regional', u'count': 2}]

The top 1st restaurant had 'None' name. It should be included since it accounts to about half of the total cuisines. Seattle seems to have a good variety of cuisines from 'pizza' to 'mexcian', 'italian' and other asian ones.

### 6. Conclusion

Seattle open street map dataset is a good data source for data wrangling purpose. It contains inconsistant street types; postcodes that are not in Seattle; and cities are in nearby cities of Seattle or even in Canada. They are all cleaned. Over view of the cleaned data was reported. Some interesting data such as top amenities and cuisine were analyzed. 

To improve the OMS data quality, I suggested to have a cross-reference with other data sources such as local goverment data, post mail company or Google Maps to ensure the correctness and consistance of the OMS data. If those cross-reference data are not available, the OMS community could collaborate and come up with their own standard data and clean it as my way in this project. 

